# Energía / Utilities: Scraping de URLs legítimas

**Fecha de ejecución:** 22/07/2025  
**Empresas objetivo:**  
Endesa, Iberdrola, Naturgy, Repsol, Cepsa, Holaluz, TotalEnergies

---


In [1]:
# Importamos las librerías necesarias para el scraping y el análisis de datos
import requests                             # Para hacer peticiones HTTP a las webs
from bs4 import BeautifulSoup               # Para parsear el HTML de la página web
from urllib.parse import urlparse, urljoin  # Para trabajar con URLs y dominios
import pandas as pd                         # Para almacenar y exportar los datos en DataFrame
import time                                 # Para pausar entre peticiones y evitar bloqueos


In [2]:
def obtener_urls(base_url, delay=1):
    """
    Descarga la página principal de una web y extrae todas las URLs internas.
    Solo añade URLs que pertenezcan al dominio principal o subdominios de la empresa.
    - base_url: URL principal de la empresa
    - delay: segundos de espera tras cada request para evitar bloqueos
    """
    try:
        # Definimos un User-Agent "realista" para que el servidor no bloquee la petición por bot
        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.124 Safari/537.36"
            )
        }
        # Hacemos la petición HTTP a la web (con timeout de 10 segundos)
        response = requests.get(base_url, timeout=10, headers=headers)

        # Parseamos el HTML recibido con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraemos el dominio principal (ej: 'amazon.es' de 'www.amazon.es')
        dominio_empresa = urlparse(base_url).netloc
        if dominio_empresa.startswith('www.'):
            dominio_base = dominio_empresa[4:]
        else:
            dominio_base = dominio_empresa

        # Creamos un set para guardar solo URLs únicas
        urls = set()

        # Buscamos todos los enlaces <a> que tengan atributo href
        for link in soup.find_all('a', href=True):
            href = link['href']                  # Obtenemos el href
            href = urljoin(base_url, href)       # Convertimos a URL absoluta
            dominio_href = urlparse(href).netloc # Dominio del enlace

            # Eliminamos 'www.' para comparar dominios correctamente
            if dominio_href.startswith('www.'):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añadimos solo si es del dominio principal o subdominio
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Esperamos el tiempo indicado (para no ser bloqueados por el servidor)
        time.sleep(delay)

        # Devolvemos la lista de URLs y un estado "OK"
        return list(urls), "OK"

    except Exception as e:
        # Si hay error, lo devolvemos como estado y lista vacía de URLs
        return [], str(e)


In [3]:
empresas_energia = {
    "Endesa": "https://www.endesa.com",
    "Iberdrola": "https://www.iberdrola.es",
    "Naturgy": "https://www.naturgy.es",
    "Repsol": "https://www.repsol.com",
    "Cepsa": "https://www.cepsa.es",
    "Holaluz": "https://www.holaluz.com",
    "TotalEnergies": "https://www.totalenergies.es"
}


In [4]:
# Lista donde almacenaremos los resultados de todas las empresas
resultados = []

# Guardamos la fecha actual 
fecha = pd.Timestamp.today().strftime("%d/%m/%Y")

# Iteramos sobre cada empresa y su URL
for nombre, url_base in empresas_energia.items():
    print(f'Scraping empresa: {nombre}')                # Log para saber el progreso
    urls, estado = obtener_urls(url_base)               # Llamamos a la función de scraping
    if urls:
        # Si hay URLs, añadimos cada una como fila en los resultados
        for url in urls:
            resultados.append({
                'empresa': nombre,
                'url': url,
                'fecha': fecha,
                'estado': estado
            })
    else:
        # Si falla o no hay URLs, registramos igualmente el intento
        resultados.append({
            'empresa': nombre,
            'url': '',
            'fecha': fecha,
            'estado': estado
        })

# Convertimos la lista de resultados en un DataFrame de pandas
df_scrap = pd.DataFrame(resultados)

# Guardamos el DataFrame como CSV en la carpeta correspondiente (cambia el nombre por sector)
df_scrap.to_csv('../data/raw/energia_legitimas_crudo.csv', index=False)

# Mostramos por pantalla las primeras filas del DataFrame para revisión rápida
print(df_scrap.head())


Scraping empresa: Endesa
Scraping empresa: Iberdrola
Scraping empresa: Naturgy
Scraping empresa: Repsol
Scraping empresa: Cepsa
Scraping empresa: Holaluz
Scraping empresa: TotalEnergies
  empresa                                                url       fecha  \
0  Endesa  https://www.endesa.com/content/endesa-com/es/p...  22/07/2025   
1  Endesa  https://www.endesa.com/es/te-ayudamos/sobre-tu...  22/07/2025   
2  Endesa  https://www.endesa.com/content/endesa-com/es/e...  22/07/2025   
3  Endesa  https://www.endesa.com/content/endesa-com/es/m...  22/07/2025   
4  Endesa  https://www.endesa.com/es/la-cara-e/endesamusi...  22/07/2025   

  estado  
0     OK  
1     OK  
2     OK  
3     OK  
4     OK  


In [5]:
print(df_scrap['empresa'].value_counts())




empresa
Repsol           257
Endesa           209
Cepsa            201
Naturgy           58
Holaluz           52
TotalEnergies     52
Iberdrola          1
Name: count, dtype: int64


| Empresa        | URLs únicas | Fecha      | Estado / Observaciones                |
|----------------|------------|------------|---------------------------------------|
| Repsol         |   257      | 22/07/2025 | OK                                    |
| Endesa         |   209      | 22/07/2025 | OK                                    |
| Cepsa          |   201      | 22/07/2025 | OK                                    |
| Naturgy        |    58      | 22/07/2025 | OK                                    |
| Holaluz        |    52      | 22/07/2025 | OK                                    |
| TotalEnergies  |    52      | 22/07/2025 | OK                                    |
| Iberdrola      |     1      | 22/07/2025 | OK – Home minimalista/poca visibilidad|


- **Empresas scrapeadas:** 7
- **Total URLs obtenidas:** 830
- **Patrones observados:**
    - Repsol, Endesa y Cepsa exponen un volumen muy alto de URLs públicas.
    - Iberdrola solo muestra 1, posiblemente por home minimalista o por estructura protegida.
    - El resto muestran un nivel medio de enlaces internos.
- **Incidencias:**
    - Ningún error crítico, todas las webs permiten scraping simple.
    - Revisar si Iberdrola ofrece secciones públicas internas interesantes (si interesa ampliar).
- **Siguiente paso:**  
    - Continuar con el sector Streaming/Entretenimiento o revisar más en profundidad webs con pocos resultados.
